In [ ]:
'''
                ******** FREE-TEXT Classification for Commit Intent Use case.
                
Two algorithm is used for model training.
a. RandomForestClassifier
b. MultiNomial Naive Bayes 

'''

In [8]:
from pandas import read_csv
import pickle
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
import nltk
from sklearn.externals import joblib
from pathlib import Path
from sklearn.metrics import accuracy_score

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\618757\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#Part-1 Load the data

filename = 'commit_intent_training_V2.csv'
names = ['commit message', 'why the commit']
df = read_csv(filename, names=names)

In [3]:
# Part-2 - Initialize the vectorizer to convert free-text into numerical representation

stemmer = PorterStemmer()
words = stopwords.words("english")
df['cleaned'] = df['commit message'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())


vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
final_features = vectorizer.fit_transform(df['cleaned']).toarray()

In [4]:
# Part-3 Split the data into training and testing dataset

X = df['cleaned']
Y = df['why the commit']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [13]:
# Part-4  train the model using MultiNomial Naive Bayes algorithm

pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1200)),
                     ('clf', MultinomialNB())])

modelnb = pipeline.fit(X_train, y_train)

ytest = np.array(y_test)
predictions =  modelrf.predict(X_test)

print("Accuracy Score of MultiNomial Naive Bayes model: %s%%" % (100*accuracy_score(ytest, predictions)))

print('\n Classification Report')
print(classification_report(ytest, predictions))
print('\n Confusion Matrix')
print(confusion_matrix(ytest, predictions))

Accuracy Score of MultiNomial Naive Bayes model: 98.10650887573964%

 Classification Report
                                       precision    recall  f1-score   support

                  adding new features       0.99      0.99      0.99       152
                           bug fixing       0.99      1.00      0.99       278
   commits that belong to no category       0.98      0.88      0.93        65
                   design improvement       0.94      0.97      0.96       102
improving non-functional requirements       0.98      0.99      0.98       248

                             accuracy                           0.98       845
                            macro avg       0.98      0.96      0.97       845
                         weighted avg       0.98      0.98      0.98       845


 Confusion Matrix
[[150   0   1   1   0]
 [  0 278   0   0   0]
 [  0   3  57   2   3]
 [  1   0   0  99   2]
 [  0   0   0   3 245]]


In [10]:
# Part-5  train the model using RandomForestClassifier algorithm


pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1200)),
                     ('clf', RandomForestClassifier())])

modelrf = pipeline.fit(X_train, y_train)
ytest = np.array(y_test)
predictions =  modelrf.predict(X_test)

print("Accuracy Score of Random Forest Classifier model: %s%%" % (100*accuracy_score(ytest, predictions)))

whycommit_prediction1 = modelrf.predict(["[Automation] - Fix test failure for test_02_revert_vm_snapshots in smoke/test_vm_snapshots.py"])
print('Commit Intent Prediction of given commit message: ', whycommit_prediction1)

Accuracy Score of Random Forest Classifier model: 98.10650887573964%
Commit Intent Prediction of given commit message:  ['bug fixing']


C:\Users\618757\AppData\Roaming\Python\Python36\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
